In [1]:
import cv2
import numpy as np

In [2]:
image = cv2.imread('digits.png')
grey = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
small = cv2.pyrDown(image)
cv2.imshow('sma',small)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
cells = [np.hsplit(row,100) for row in np.vsplit(grey,50)]

In [4]:
x = np.array(cells)

In [5]:
train = x[:,:70].reshape(-1,400).astype(np.float32)
test = x[:,70:100].reshape(-1,400).astype(np.float32)

In [6]:
k = list(range(10))

In [7]:
train_label = np.repeat(k,350)
train_label_1 = np.repeat(k,350)[:,np.newaxis]

In [8]:
train_label

array([0, 0, 0, ..., 9, 9, 9])

In [9]:
train_label_1

array([[0],
       [0],
       [0],
       ...,
       [9],
       [9],
       [9]])

In [10]:
from sklearn.neighbors import KNeighborsClassifier

In [11]:
knn = KNeighborsClassifier(n_neighbors= 3)
knn.fit(train,train_label)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [13]:
def x_sort(cnt):
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    return cx

In [14]:
def make_square(not_square):
    img_dim = not_square.shape
    height = img_dim[0]
    width = img_dim[1]
    if height == width:
        return not_square
    else:
        if height > width:
            pad = (height - width)/2
            square = cv2.copyMakeBorder(not_square,0,0,int(pad),int(pad),cv2.BORDER_CONSTANT,value = [0,0,0])
        else:
            pad = (width - height)/2
            square = cv2.copyMakeBorder(not_square,int(pad),int(pad),0,0,cv2.BORDER_CONSTANT,value = [0,0,0])
        return square

In [15]:
def resize_to_pixel(dimensions, image):
    # This function then re-sizes an image to the specificied dimenions
    
    buffer_pix = 4
    dimensions  = dimensions - buffer_pix
    squared = image
    r = float(dimensions) / squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    width_r = img_dim2[1]
    BLACK = [0,0,0]
    if (height_r > width_r):
        resized = cv2.copyMakeBorder(resized,0,0,0,1,cv2.BORDER_CONSTANT,value=BLACK)
    if (height_r < width_r):
        resized = cv2.copyMakeBorder(resized,1,0,0,0,cv2.BORDER_CONSTANT,value=BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized,p,p,p,p,cv2.BORDER_CONSTANT,value=BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Padded Height = ", height, "Width = ", width)
    return ReSizedImg

In [16]:
import cv2
import numpy as np

image = cv2.imread('numbers.jpg')
grey = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(grey, (5,5),0)
edge = cv2.Canny(blur,30,150)
_,contours,_ = cv2.findContours(edge.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
contours = sorted(contours,key = x_sort, reverse=False)

full_numbers = []

for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    roi = blur[int(y):int(y+h),int(x):int(x+w)]
    _,roi = cv2.threshold(roi,127,255,cv2.THRESH_BINARY_INV)
    square = make_square(roi)
    final = resize_to_pixel(20,square)
    final_vector = final.reshape((1,400)).astype(np.float32)
    result = knn.predict(final_vector)
    full_numbers.append(str(int(result)))
    cv2.rectangle(image,(int(x),int(y)),(int(x+w),int(y+h)),(0,0,255),2)
    cv2.putText(image,str(int(result)),(int(x),int(y+50)),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
    cv2.imshow('image',image)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()
    

ValueError: cannot reshape array of size 380 into shape (1,400)